In [1]:
import tensorflow as tf
import keras
import os
from tensorflow.keras.backend import set_session

os.environ['CUDA_VISIBLE_DEVICES'] = "1"
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
import torch
import torch.nn
import numpy as np
import matlab
import matlab.engine
import h5py
import sys

In [3]:
sys.path.append('/cluster/sj1/bb_opt/src')

In [4]:
from chemvae_bopt import chemvae_gpopt_pes
import gp_acquisition as ack

In [5]:
zinc250k = []
with open('/cluster/sj1/bb_opt/chemical_vae/models/zinc_properties/250k_rndm_zinc_drugs_clean_3.csv') as f:
    next(f)
    for line in f:
        line = [k.strip() for k in line.strip().split('\t')]
        zinc250k += [[line[0], [float(k) for k in line[1:]]]]

In [6]:
from chemvae_keras import vae_utils
from chemvae_keras import mol_utils as mu

In [7]:
is_cuda = True
directory = '../chemical_vae/models/zinc_properties'
n_init_samples = 1000
n_opt_samples = 10
cur_dir = '/cluster/sj1/bb_opt/src'
score_fn = lambda x : x[0]

In [8]:
def obj(x):
    noise = 5
    df = vae.z_to_smiles(x, decode_attempts=100, noise_norm=noise)
    smiles = {k : vae.smiles_to_hot(mu.canon_smiles(k)) for k in set(df['smiles'])}
    best_smile = None
    best_smile_score = -1
    for smile in smiles:
        score = score_fn(vae.predict_prop_Z(vae.encode(smile))[0])
        if score > best_smile_score:
            best_smile = smile
            best_smile_score = score
    return best_smile_score

In [9]:
os.chdir(cur_dir)
vae = vae_utils.VAEUtils(directory=directory)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


From /cluster/sj1/anaconda2/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1208: calling reduce_prod (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
/cluster/sj1/anaconda2/envs/py36/lib/python3.6/site-packages/keras/models.py:258: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Instructions for updating:
keep_dims is deprecated, use keepdims instead


From /cluster/sj1/anaconda2/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1190: calling reduce_sum (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


From /cluster/sj1/anaconda2/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1154: calling reduce_max (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Using standarized functions? True
Standarization: estimating mu and std values ...done!


In [10]:
zinc250k.sort(key=lambda k: score_fn(k[1]))

In [11]:
smiles = []
i = 0
for smiles_string, props in zinc250k[:2*n_init_samples]:
    i += 1
    if i % 10000 == 0:
        print("done {:d}K samples".format(i//1000))
    smiles += [vae.smiles_to_hot(mu.canon_smiles(smiles_string), canonize_smiles=True)]

In [12]:
smiles[0].shape

(1, 120, 35)

In [13]:
x_samples = [vae.encode(k)[0] for k in smiles[:n_init_samples]]

In [14]:
predictor_model = lambda x : score_fn(vae.predict_prop_Z(x[np.newaxis, :])[0])

In [15]:
predictor_model(x_samples[0])

0.5949456

In [16]:
nvars = x_samples[0].shape[-1]
x_min = matlab.double([-1]*nvars)
x_max = matlab.double([1]*nvars)

In [17]:
eng = matlab.engine.start_matlab()

In [25]:
temp = [predictor_model(k) for k in x_samples]

In [31]:
y_samples = eng.transpose(matlab.double([predictor_model(k) for k in x_samples]))

In [19]:
x_samples_matlab = matlab.double([k.tolist() for k in x_samples])

In [21]:
eng.size(x_samples_matlab)

matlab.double([[1000.0,196.0]])

In [32]:
eng.size(y_samples)

matlab.double([[1000.0,1.0]])

In [27]:
eng.size(matlab.double(temp, size=(len(temp),)))

matlab.double([[1.0,1000.0]])

In [33]:
x_samples_matlab, y_samples, l, sigma, sigma0 = ack.init_mes(eng, x_samples_matlab, y_samples)

TypeError: 'NoneType' object is not iterable

In [ ]:
print "done"

In [ ]:
import gp_acquisition
gp_acquisition.init_mes()